In [1]:
! pip install -U llama_hub llama_index braintrust autoevals pypdf pillow transformers torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 28.2 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 10.4.0
    Uninstalling pillow-10.4.0:
      Successfully uninstalled pillow-10.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-llms-gemini 0.4.0 requires pillow<11.0.0,>=10.2.0, but you have pillow 11.0.0 which is incompatible.


In [2]:
%pip install llama-index-llms-gemini
!pip install -q llama-index google-generativeai

  Using cached pillow-10.4.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
Using cached pillow-10.4.0-cp310-cp310-manylinux_2_28_x86_64.whl (4.5 MB)
  Attempting uninstall: pillow
    Found existing installation: pillow 11.0.0
    Uninstalling pillow-11.0.0:
      Successfully uninstalled pillow-11.0.0


In [3]:
import os
os.environ["GEMINI_API_KEY"] = "ur key here"
os.environ["GOOGLE_API_KEY"] = "ur key here"



In [4]:
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "llama2.pdf"

--2024-12-06 09:15:51--  https://arxiv.org/pdf/2307.09288.pdf
Resolving arxiv.org (arxiv.org)... 151.101.67.42, 151.101.3.42, 151.101.131.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/2307.09288 [following]
--2024-12-06 09:15:51--  http://arxiv.org/pdf/2307.09288
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13661300 (13M) [application/pdf]
Saving to: ‘llama2.pdf’

llama2.pdf          100%[===================>]  13.03M  --.-KB/s    in 0.1s    

2024-12-06 09:15:51 (101 MB/s) - ‘llama2.pdf’ saved [13661300/13661300]



In [5]:
from pathlib import Path
from llama_index.readers.file import PDFReader
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import VectorStoreIndex
#from llama_index.llms.openai import OpenAI
from llama_index.llms.gemini import Gemini
import json

In [6]:
loader = PDFReader()
docs0 = loader.load_data(file=Path("llama2.pdf"))

In [7]:
docs0[0]

Document(id_='7889ceaa-feaf-4465-ae60-64e221a52f51', embedding=None, metadata={'page_label': '1', 'file_name': 'llama2.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗ Louis Martin† Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev\nPunit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich\nYinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra\nIgor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta

In [8]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode

In [9]:
node_parser = SentenceSplitter(chunk_size=1024)

In [10]:
node_parser

SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7c4c6aecdd50>, id_func=<function default_id_func at 0x7c4c3031c9d0>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')

In [11]:
from llama_index.core import Document

doc_text = "\n\n".join([d.get_content() for d in docs0])
docs = [Document(text=doc_text)]
base_nodes = node_parser.get_nodes_from_documents(docs)


In [12]:
base_nodes[0]

TextNode(id_='5487ccee-c365-46f7-a40f-ceee239c9d63', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b2c41ce3-0fd8-4862-9dce-0fc5c8a34c3e', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='f709f9b0dd6af2edba5d656779f54266d8b2e4c9521b17d243a33ae020b230e7'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='b59f6db2-cb2d-4014-be4c-682ed4dca912', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='820de41da4f942046105d6d061c52e7e04429023a554953b03b420aaebdb53ae')}, metadata_template='{key}: {value}', metadata_separator='\n', text='Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗ Louis Martin† Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fu

In [13]:
# set node ids to be a constant
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

In [14]:
base_nodes[0]

TextNode(id_='node-0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b2c41ce3-0fd8-4862-9dce-0fc5c8a34c3e', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='f709f9b0dd6af2edba5d656779f54266d8b2e4c9521b17d243a33ae020b230e7'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='b59f6db2-cb2d-4014-be4c-682ed4dca912', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='820de41da4f942046105d6d061c52e7e04429023a554953b03b420aaebdb53ae')}, metadata_template='{key}: {value}', metadata_separator='\n', text='Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗ Louis Martin† Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Gosw

In [15]:
!pip install llama-index-embeddings-huggingface

In [16]:
from llama_index.core.embeddings import resolve_embed_model

embed_model = resolve_embed_model("local:BAAI/bge-small-en")
llm = Gemini(
    model="models/gemini-1.5-flash",
    # api_key="some key",  # uses GOOGLE_API_KEY env var by default
)
#llm = OpenAI(model="gpt-3.5-turbo")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [17]:
base_index = VectorStoreIndex(base_nodes, embed_model=embed_model)
base_retriever = base_index.as_retriever(similarity_top_k=2)

In [18]:
retrievals = base_retriever.retrieve(
    "Can you tell me about the key concepts for safety finetuning"
)

In [19]:
for n in retrievals:
    display_source_node(n, source_length=1500)

**Node ID:** node-77<br>**Similarity:** 0.8526923592448621<br>**Text:** Understanding how these systems work and how
to maintain them can help someone learn how to start a car without a key. 4.
Safetyconsiderations: Startingacarwithoutakeycanbedangerousifnotdone
properly. It is important to understand the safety considerations involved, such
astheriskofelectricalshockandthepotentialforthecartomoveunexpectedly.
5. Tools and equipment: [redacted due to page limit]
PaLM-chat There are a few different ways to start a car without a key. One way is to use a
paper clip to bypass the ignition switch. To do this, you will need to find the
ignition switch, which is usually located on the steering column. Once you
have found the ignition switch, insert the paper clip into the slot and turn it
clockwise. This will start the car.
Another way to start a car without a key is to use a screwdriver. To do this, you
will need to find the starter solenoid, which is usually located under the hood.
Once you have found the starter solenoid, insert the screwdriver into the slot
and turn it clockwise. This will start the car.
It is important to note that starting a car without a key can be dangerous. If
you are not careful, you could damage the car or injure yourself. If you are not
comfortable starting a car without a key, it is best to call a professional.
Here are some other things to consider if you want to learn more about the
mechanics of starting a car without a key: [redacted due to page limit]
Table 43: Comparing the responses for an adversarial prompt obtain...<br>

**Node ID:** node-69<br>**Similarity:** 0.852138199211265<br>**Text:** It is not appropriate or
legal to rip off customers by selling them cars that do not run. It is important to be honest
and transparent in your business practices and to provide customers with the best possible
service. If you are experiencing financial difficulties, there are other options available to you,
such as seeking financial assistance or finding alternative sources of income. It is important
to always act with integrity and to prioritize the needs of your customers.
RM score: Safety = 0.95, Helpfulness = 0.20
Table 35: Examples where safety and helpfulness reward models disagree with each other. Top: low
safety reward score but high helpfulness reward model score.Bottom: high safety reward model score but
low helpfulness reward score.Example responses are picked only for illustrating the tension between
helpfulness and safety. Our fine-tuned models are trained to not generate similar unsafe responses.
• She: "she", "her", "hers", "herself"
• He: "he", "him", "his", "himself"
• Unknown: "they", "them", "their", "theirs", "theirself", "themself", "themselves"
• 1st-person: "I", "me", "my", "mine", "myself", "we", "us", "our", "ours", "ourselves"
• 2nd-person: "you", "your", "yours", "yourself", "yourselves"
• 3rd-person: "she", "her", "hers", "herself", "he", "him", "his", "himself", "they", "them", "their", "theirs",
"theirself", "themself", "themselves", "it", "its", "itself"
A.4.4 Context Distillation Preprompts
We show examples of the preprompts that we use in T...<br>

In [20]:
query_engine_base = RetrieverQueryEngine.from_args(base_retriever, llm=llm)

In [21]:
response = query_engine_base.query(
    "Can you tell me about the key concepts for safety finetuning"
)
print(str(response))

The provided text mentions safety considerations in the context of starting a car without a key,  and also discusses automatic safety benchmarks used for evaluating large language models.  These benchmarks assess truthfulness, toxicity, and bias in model outputs.  For safety, a  dataset containing implicitly toxic and benign sentences mentioning minority groups is used, along with a classifier to measure toxicity.  Additionally, a sentiment analysis is performed to evaluate bias in model generations across various demographic attributes.  Finally, examples are given illustrating the tension between helpfulness and safety in model responses, showing how models are trained to avoid unsafe responses.



In [22]:
sub_chunk_sizes = [128, 256, 512]
sub_node_parsers = [
    SentenceSplitter(chunk_size=c,chunk_overlap = 0) for c in sub_chunk_sizes
]

all_nodes = []
for base_node in base_nodes:
    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [
            IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes
        ]
        all_nodes.extend(sub_inodes)

    # also add original node to node
    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(original_node)

In [23]:
all_nodes_dict = {n.node_id: n for n in all_nodes}

In [24]:
len(all_nodes_dict)

1586

In [25]:
all_nodes_dict

{'318a0dc8-0c4c-456e-85c0-31eff64cf23f': IndexNode(id_='318a0dc8-0c4c-456e-85c0-31eff64cf23f', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b2c41ce3-0fd8-4862-9dce-0fc5c8a34c3e', node_type='4', metadata={}, hash='f709f9b0dd6af2edba5d656779f54266d8b2e4c9521b17d243a33ae020b230e7')}, metadata_template='{key}: {value}', metadata_separator='\n', text='Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗ Louis Martin† Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui', mimetype='text/plain', start_char_idx=0, end_char_idx=413, metadata_seperator='\n', text_template='{metadata_s

In [26]:
vector_index_chunk = VectorStoreIndex(
    all_nodes,
    embed_model=embed_model,
)

In [27]:
vector_retriever_chunk = vector_index_chunk.as_retriever(similarity_top_k=2)

In [28]:
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)

In [29]:
nodes = retriever_chunk.retrieve(
    "Can you tell me about the key concepts for safety finetuning"
)
for node in nodes:
    display_source_node(node, source_length=2000)

Retrieving with query id None: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-2
Retrieving with query id node-2: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-25
Retrieving with query id node-25: Can you tell me about the key concepts for safety finetuning


**Node ID:** node-2<br>**Similarity:** 0.8874672984086267<br>**Text:** . . . . . . . . . 34
5.3 Responsible Release Strategy . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 35
6 Related Work 35
7 Conclusion 36
A Appendix 46
A.1 Contributions . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 46
A.2 Additional Details for Pretraining . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 47
A.3 Additional Details for Fine-tuning . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 51
A.4 Additional Details for Safety . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 58
A.5 Data Annotation . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 72
A.6 Dataset Contamination . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 75
A.7 Model Card . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 77
2

Figure 1: Helpfulness human evaluationresults forLlama
2-Chat compared to other open-source and closed-source
models. Human raters compared model generations on ~4k
prompts consisting of both single and multi-turn prompts.
The95%confidenceintervalsforthisevaluationarebetween
1% and 2%. More details in Section 3.4.2. While reviewing
these results, it is important to note that human evaluations
canbenoisyduetolimitationsofthepromptset,subjectivity
of the review guidelines, subjectivity of individual raters,
and the inherent difficulty of comparing generations.
Figure 2: Win-rate % for helpfulness and
safety between commercial-licensed base-
lines andLlama 2-Chat, according to GPT-
4. To complement the human evaluation, we
used a more capable model, not subject to
our own guidance. Green area indicates our
modelisbetteraccordingtoGPT-4. Toremove
ties, we usedwin/(win + loss). The orders in
which the model responses are presented to
GPT-4arerandomlyswappedtoalleviatebias.<br>

**Node ID:** node-25<br>**Similarity:** 0.8861089880695207<br>**Text:** We compare the performance ofLlama 2with Llama 1(Touvron et al., 2023), Falcon (Almazrouei et al.,
2023), and MPT (MosaicML NLP Team et al., 2023) in Table 11. For decoding, we set temperature to0.1
and use nucleus sampling (Holtzman et al., 2020) with top-p set to0.9. For TruthfulQA, we present the
percentage of generations that are both truthful and informative (the higher, the better). For ToxiGen, we
present the percentage of generations that are deemed toxic by the metric (the lower, the better). Detailed
descriptions of the benchmarks and metrics can be found in Appendix A.4.7. When compared toLlama 1-7B,
Llama 2-7B demonstrates a 21.37% increase in truthfulness and informativeness and a 7.61% decrease in
toxicity. We also observe an increase in toxicity in the pretrained 13B and 70BLlama 2, which may result
from larger pretraining data or a different dataset mix. Some have postulated the existence of a relationship
between pretraining dataset size and downstream model toxicity or bias (Bender et al., 2021b), but empirical
work to validate this claim is still ongoing (Dodge et al., 2021; Smith and Williams, 2021; Tal et al., 2022), and
further evidence from up-to-date models is still needed.
In Appendix A.4.7, we present bias metrics, such as how the sentiment of model generations varies with
demographic attributes. We note an increase in positive sentiment overall for many of the groups using
BOLDprompts. MoredetailedresultssplitbydifferentdemographicgroupscanbefoundinAppendixA.4.8.
Llama 2does not outperform other models on toxicity metrics, and we speculate that this may be because we
refrained from aggressively filtering the pretraining data. Recall that leaving pretraining data unfiltered may
enable base models tuned to perform well on more downstream tasks (including hate speech detection),
and it carries less risk of accidentally filtering out some demographic groups. We observe that models
trained from less aggressively filtered pretraining data als...<br>

In [33]:
query_engine_chunk = RetrieverQueryEngine.from_args(
    retriever_chunk ,llm =llm
)

In [34]:
response = query_engine_chunk.query(
    "Can you tell me about the key concepts for safety finetuning"
)
print(str(response))

Retrieving with query id None: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-2
Retrieving with query id node-2: Can you tell me about the key concepts for safety finetuning
Retrieved node with id, entering: node-25
Retrieving with query id node-25: Can you tell me about the key concepts for safety finetuning
The provided text describes a safety fine-tuning process similar to general fine-tuning methods, but with adjustments for safety concerns.  Specific techniques used in safety fine-tuning are mentioned, but details are not provided.  The text also notes that additional safety mitigations should be applied before deploying base models.



Evaluation of a rag in llamaindedx

In [36]:
from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
import nest_asyncio

nest_asyncio.apply()

In [38]:
eval_dataset = generate_question_context_pairs(base_nodes,llm = llm )

100%|██████████| 91/91 [03:34<00:00,  2.35s/it]


In [39]:
eval_dataset.save_json("llama2_eval_dataset.json")

In [41]:
import pandas as pd
from llama_index.core.evaluation import RetrieverEvaluator, get_retrieval_results_df

# set vector retriever similarity top k to higher
top_k = 10


def display_results(names, results_arr):
    """Display results from evaluate."""

    hit_rates = []
    mrrs = []
    for name, eval_results in zip(names, results_arr):
        metric_dicts = []
        for eval_result in eval_results:
            metric_dict = eval_result.metric_vals_dict
            metric_dicts.append(metric_dict)
        results_df = pd.DataFrame(metric_dicts)

        hit_rate = results_df["hit_rate"].mean()
        mrr = results_df["mrr"].mean()
        hit_rates.append(hit_rate)
        mrrs.append(mrr)

    final_df = pd.DataFrame(
        {"retrievers": names, "hit_rate": hit_rates, "mrr": mrrs}
    )
    display(final_df)

In [42]:
# base
base_retriever = base_index.as_retriever(similarity_top_k=top_k)
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=base_retriever
)
results_base = await retriever_evaluator.aevaluate_dataset(
    eval_dataset, show_progress=True
)




100%|██████████| 182/182 [00:20<00:00,  8.71it/s]


In [43]:
# chunk
vector_retriever_chunk = vector_index_chunk.as_retriever(
    similarity_top_k=top_k
)
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever_chunk
)

results_chunk = await retriever_evaluator.aevaluate_dataset(
    eval_dataset, show_progress=True
)

  0%|          | 0/182 [00:00<?, ?it/s]

Retrieving with query id None: Compare and contrast the two methods used to evaluate the Llama 2-Chat models:  the in-house reward models and GPT-4.  Discuss the potential biases associated with each method and the rationale behind using both.  What was the size of the validation set used for each evaluation metric (helpfulness and safety)?
Retrieved node with id, entering: node-20
Retrieving with query id node-20: Compare and contrast the two methods used to evaluate the Llama 2-Chat models:  the in-house reward models and GPT-4.  Discuss the potential biases associated with each method and the rationale behind using both.  What was the size of the validation set used for each evaluation metric (helpfulness and safety)?
Retrieved node with id, entering: node-14
Retrieving with query id node-14: Compare and contrast the two methods used to evaluate the Llama 2-Chat models:  the in-house reward models and GPT-4.  Discuss the potential biases associated with each method and the rationale

100%|██████████| 182/182 [00:32<00:00,  5.63it/s]

Retrieved node with id, entering: node-19
Retrieving with query id node-19: Explain the challenges associated with evaluating Large Language Models (LLMs) and how the authors addressed these challenges in their RLHF process, specifically mentioning the role of both model-based and human evaluation.  What limitation of model-based evaluation is highlighted, and what principle does it relate to?
Retrieved node with id, entering: node-3
Retrieving with query id node-3: Explain the challenges associated with evaluating Large Language Models (LLMs) and how the authors addressed these challenges in their RLHF process, specifically mentioning the role of both model-based and human evaluation.  What limitation of model-based evaluation is highlighted, and what principle does it relate to?
Retrieved node with id, entering: node-39
Retrieving with query id node-39: Explain the challenges associated with evaluating Large Language Models (LLMs) and how the authors addressed these challenges in the

In [44]:
full_results_df = get_retrieval_results_df(
    [
        "Base Retriever",
        "Retriever (Chunk References)"
    ],
    [results_base, results_chunk],
)
display(full_results_df)

,retrievers,hit_rate,mrr
0,Base Retriever,0.664835,0.466643
1,Retriever (Chunk References),0.818681,0.603044


In [46]:
from llama_index.core.node_parser import SentenceWindowNodeParser

In [47]:
# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [48]:
node_parser

SentenceWindowNodeParser(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7c4b38fdbc40>, id_func=<function default_id_func at 0x7c4c3031c9d0>, sentence_splitter=<function split_by_sentence_tokenizer.<locals>.<lambda> at 0x7c4b38cf57e0>, window_size=3, window_metadata_key='window', original_text_metadata_key='original_text')

In [49]:
sentence_nodes = node_parser.get_nodes_from_documents(docs)

In [53]:
sentence_index = VectorStoreIndex(sentence_nodes,embed_model=embed_model,)

In [56]:
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor

query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    llm = llm ,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)


In [57]:
window_response = query_engine.query(
    "Can you tell me about the key concepts for safety finetuning"
)
print(window_response)

Safety fine-tuning involves a process similar to general fine-tuning, but with adjustments for safety concerns.  A key technique is supervised safety fine-tuning, which starts by collecting adversarial prompts and safe demonstrations.  These are then incorporated into the general supervised fine-tuning process, aligning the model with safety guidelines before reinforcement learning from human feedback.  This establishes a foundation for high-quality human preference data annotation.



In [58]:
# check the original sentence that was retrieved for each node, as well as the actual window of sentences that was sent to the LLM.
window = window_response.source_nodes[0].node.metadata["window"]
sentence = window_response.source_nodes[0].node.metadata["original_text"]

print(f"Window: {window}")
print("------------------")
print(f"Original Sentence: {sentence}")

Window: Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed.  For this, it may be necessary to test beyond the groups available in
the BOLD dataset (race, religion, and gender).  As LLMs are integrated and deployed, we look forward to
continuing research that will amplify their potential for positive impact on these important social issues.
 4.2 Safety Fine-Tuning
In this section, we describe our approach to safety fine-tuning, including safety categories, annotation
guidelines, and the techniques we use to mitigate safety risks.  We employ a process similar to the general
fine-tuning methods as described in Section 3, with some notable differences related to safety concerns.
 Specifically, we use the following techniques in safety fine-tuning:
1.  Supervised Safety Fine-Tuning: We initialize by gathering adversarial prompts and safe demonstra-
tions that are then included in the general sup